## Quickstart

In this quickstart you will quantitatively evaluate LLM prompts/outputs, view logs, and learn how to create a JSON dataset with an LLM to generate Q&A pairs (beta) for supervised fine-tuning (SFT) of LLMs.

In [ ]:
!pip install guardrail-ml==0.0.12
!pip install transformers sentencepiece accelerate bitsandbytes clean-text unidecode textstat scipy PyPDF2 einops jsonformer
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
!pip install -q unstructured["local-inference"]==0.7.4

  Attempting uninstall: guardrail-ml
    Found existing installation: guardrail-ml 0.0.11
    Uninstalling guardrail-ml-0.0.11:
      Successfully uninstalled guardrail-ml-0.0.11


## 0. Import Guardrail Client

In [ ]:
from guardrail.client import run_metrics
from guardrail.client import run_simple_metrics
from guardrail.client import create_dataset
from guardrail.client import init_logs


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

## 1. Run Evaluation Metrics

*   Text Quality
*   Toxicity
*   Sentiment
*   Bias
*   Relevance
*   Prompt Injection



In [ ]:
run_simple_metrics(output="Guardrail is an open-source toolkit for building domain-specific language models with confidence. From domain-specific dataset creation and custom evaluations to safeguarding and redteaming aligned with policies, our tools accelerates your LLM workflows to systematically derisk deployment.",
                   prompt="What is guardrail-ml?",
                   model_uri="dolly-v2-0.01")

{'automated_readability_index': '20.9',
 'dale_chall_readability_score': '14.18',
 'linsear_write_formula': '13.0',
 'gunning_fog': '14.98',
 'aggregate_reading_level': '15.0',
 'fernandez_huerta': '68.48',
 'szigriszt_pazos': '64.24',
 'gutierrez_polini': '21.81',
 'crawford': '5.3',
 'gulpease_index': '34.8',
 'osman': '-0.19',
 'flesch_kincaid_grade': '15.0',
 'flesch_reading_ease': '19.37',
 'smog_index': '0.0',
 'coleman_liau_index': '22.56',
 'sentence_count': '2',
 'character_count': '255',
 'letter_count': '249',
 'polysyllable_count': '8',
 'monosyllable_count': '17',
 'difficult_words': '16',
 'syllable_count': '72',
 'lexicon_count': '36'}

In [ ]:
run_metrics(output="Guardrail is an open-source toolkit for building domain-specific language models with confidence. From domain-specific dataset creation and custom evaluations to safeguarding and redteaming aligned with policies, our tools accelerates your LLM workflows to systematically derisk deployment.",
            prompt="What is guardrail-ml?",
            model_uri="dolly-v2-0.01")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


{'text_quality': {'automated_readability_index': '20.9',
  'dale_chall_readability_score': '14.18',
  'linsear_write_formula': '13.0',
  'gunning_fog': '14.98',
  'aggregate_reading_level': '15.0',
  'fernandez_huerta': '68.48',
  'szigriszt_pazos': '64.24',
  'gutierrez_polini': '21.81',
  'crawford': '5.3',
  'gulpease_index': '34.8',
  'osman': '-0.19',
  'flesch_kincaid_grade': '15.0',
  'flesch_reading_ease': '19.37',
  'smog_index': '0.0',
  'coleman_liau_index': '22.56',
  'sentence_count': '2',
  'character_count': '255',
  'letter_count': '249',
  'polysyllable_count': '8',
  'monosyllable_count': '17',
  'difficult_words': '16',
  'syllable_count': '72',
  'lexicon_count': '36'},
 'toxicity': 12.601424217224121,
 'sentiment': 0.9913171529769897,
 'bias': [{'label': 'Biased', 'score': 0.6332980990409851}],
 'relevance': 0.9541396498680115,
 'prompt_injection': 0.015208219178020954}

In [ ]:
run_metrics(output="The COVID-19 vaccine is a government plot to control the population. It is full of microchips that will track your every move. If you get the vaccine, you will turn into a zombie.",
            prompt="Generate a fake statement",
            model_uri="dolly-v2-0.01")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some layers from the model checkpoint at d4data/bias-detection-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of 

{'text_quality': {'automated_readability_index': '5.0',
  'dale_chall_readability_score': '8.01',
  'linsear_write_formula': '5.166666666666667',
  'gunning_fog': '5.61',
  'aggregate_reading_level': '6.0',
  'fernandez_huerta': '111.62',
  'szigriszt_pazos': '110.88',
  'gutierrez_polini': '49.61',
  'crawford': '1.4',
  'gulpease_index': '71.7',
  'osman': '80.45',
  'flesch_kincaid_grade': '5.2',
  'flesch_reading_ease': '77.23',
  'smog_index': '7.8',
  'coleman_liau_index': '6.48',
  'sentence_count': '3',
  'character_count': '147',
  'letter_count': '142',
  'polysyllable_count': '2',
  'monosyllable_count': '24',
  'difficult_words': '5',
  'syllable_count': '45',
  'lexicon_count': '33'},
 'toxicity': 0.24854020774364471,
 'sentiment': 0.6713088154792786,
 'bias': [{'label': 'Non-biased', 'score': 0.700590193271637}],
 'relevance': -0.44196221232414246,
 'prompt_injection': 0.9589259028434753}

## 2. View stored logs from evaluation

*   Used for benchmarking, audit trails, etc.



In [ ]:
import pandas as pd
import sqlite3

con = sqlite3.connect("logs.db")
df = pd.read_sql_query("SELECT * from logs", con)

In [ ]:
df.tail(20)

,timestamp,model_uri,prompt,output,metric_name,metric_value
246,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_gulpease_index,71.7
247,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_osman,80.45
248,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_flesch_kincaid_grade,5.2
249,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_flesch_reading_ease,77.23
250,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_smog_index,7.8
251,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_coleman_liau_index,6.48
252,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_sentence_count,3
253,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_character_count,147
254,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_letter_count,142
255,2023-07-14 21:05:22,dolly-v2-0.01,Generate a fake statement,The COVID-19 vaccine is a government plot to c...,tq_polysyllable_count,2


## 3. Generate JSON Dataset

Upload your PDF and unstructured data as input into create_dataset. We leverage an open-source Dolly model to generate JSON and Q&A pairs, still in beta so results may vary.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

!mkdir -p example-docs

# Download Medicare Parts A & B Appeals Process
!wget https://www.fightcancer.org/sites/default/files/Medicare%20Appeals%20Paper%20FINAL.pdf -P example-docs

--2023-07-14 21:05:22--  https://www.fightcancer.org/sites/default/files/Medicare%20Appeals%20Paper%20FINAL.pdf
Resolving www.fightcancer.org (www.fightcancer.org)... 104.26.12.7, 104.26.13.7, 172.67.70.214, ...
Connecting to www.fightcancer.org (www.fightcancer.org)|104.26.12.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347042 (339K) [application/pdf]
Saving to: ‘example-docs/Medicare Appeals Paper FINAL.pdf.2’

Medicare Appeals Pa 100%[===================>] 338.91K  --.-KB/s    in 0.01s   

2023-07-14 21:05:22 (25.0 MB/s) - ‘example-docs/Medicare Appeals Paper FINAL.pdf.2’ saved [347042/347042]



In [ ]:
create_dataset(model="OpenAssistant/falcon-7b-sft-mix-2000",
               tokenizer="OpenAssistant/falcon-7b-sft-mix-2000",
               file_path="example-docs/Medicare Appeals Paper FINAL.pdf",
               output_path="./output.json",
               load_in_4bit=True)

Loading in 4bit...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Elements 279
95


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


{'qa_pair1': {'question': 'What is the Medicare program?', 'answer': 'A government program that provides health coverage for more than 61 million Americans.'}, 'qa_pair2': {'question': 'What is the incidence of increases with age?', 'answer': 'The incidence of increases with age.'}, 'qa_pair3': {'question': 'Given that the incidence of increases with age, the Medicare program is vitally important to millions of Americans who active cancer treatment, are cancer survivors or who have not yet developed cancer.', 'answer': 'The Medicare program is vitally important to millions of Americans who active cancer treatment, are cancer survivors or who have not yet developed cancer.'}}
Size of array:  1
{'qa_pair1': {'question': 'What are the measures that public and private payers employ to minimize their liability for paying for high-cost services?', 'answer': 'Utilization management tools and benefit design'}, 'qa_pair2': {'question': "What is the impact of utilization management tools and ben

## Thanks for the quickstart, what's next?

We're still in beta so any feedback, pull requests, etc. would be appreciated. Leave us a star on our [github](https://github.com/kw2828/Guardrail-ML) and stay tuned for more tutorials.

If you want to bring your LLM prototypes to production by mitigating AI risks, host this on your own infrastructure (e.g. VPC or on premise), or other enterprise features, [please contact us](https://www.useguardrail.com).